# Urls

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install cchardet
!pip install requests-futures
!pip install http_request_randomizer
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
req_proxy = RequestProxy()
proxy_list = req_proxy.get_proxy_list()

In [3]:
import pandas as pd
name = '2021-04-15__9.csv'
table = pd.read_csv('/content/drive/MyDrive/Project/Data/zip/' + name)

# Scraping

In [ ]:
from requests import Session
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from os.path import join
import re
import pandas as pd
import time
import cchardet
import lxml
from numpy import random

http_proxy  = None
https_proxy = None
proxy = None
proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
            }
start = time.time()
k = False
fs = FuturesSession()
sess = Session()
r = fs.get("https://www.kickstarter.com").result()
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]
headers= {
              "x-csrf-token": xcsrf
          }
query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

skipped = []
used_proxies = []
last_5 = []
c= 0
old_cols = table.columns.to_list() 
scrape_cols = ['image', 'has_video', 'n_tiers', 'tiers_values', 'n_images', ' n_gifs', 
                             'n_websites', 'fb_linked', 'n_collab', 'collab_names']
cols = old_cols + scrape_cols
df = pd.DataFrame(columns=cols)
for i, u in enumerate(table['urls']):
  url = eval(u)['web']['project']
  succesful = False
  page = time.time()
  slug = re.search('/projects/(.*)\?', url).group(1)
  url = re.search('(.*)\?', url).group(1)
  print(f"------ Page {i+1}: {slug} ------")

  while not succesful:
    try:
      rs = [fs.get(url), fs.post("https://www.kickstarter.com/graph", proxies = proxyDict,
          headers=headers,
          json = {
              "operationName":"Campaign",
              "variables":{
                  "slug": slug
              },
              "query": query
          })]

      time.sleep(1)
      r = None
      r = sess.get(url + '/creator_bio')

      soup = BeautifulSoup(r.content, 'lxml')
      ## Websites
      websites = 0
      if soup.find("ul", {'class':'links list f5 bold'}):
        websites = len(soup.find("ul", {'class':'links list f5 bold'}).find_all('li'))
          
      ## Bio
      bio = soup.find('div', {'class': 'readability'})
      if bio:
        bio_text = []
        for p in bio.find_all('p'):
            if p.find(text=True) != None:
                bio_text.append(p.find(text=True))
            
        bio_text = ' '.join(bio_text)
      else:
        bio_text = None
      ## Facebook
      fb = soup.find("div", {'class':"facebook py2 border-bottom f5"}) 
      fb_linked = fb.find("a", {'class': 'popup'}) != None
      
      # Collaborators
      n_collab = 0
      collaborators = None
      if soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}):
        collaborators = soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}).findChildren('a')
        n_collab = len(collaborators)
      
      # names
      collab_names = []
      if collaborators:
        for col in collaborators:
            collab_names.append(re.search(r'/profile/(.*?)/about', str(col)).group(1))   

      r = None
      r = rs[0].result()
      soup = BeautifulSoup(r.content, 'lxml')
      
      ## Image
      image = soup.find('img')['src']
      ## Video
      has_video = soup.find('video') is not None
      ## Pledge tiers
      tiers = soup.find_all("div", {"class": "pledge__info"})
      tiers_values = []
      for tier in tiers:
          s = str(tier.find("span", {"class": "money"}))
          if re.search(r'\d+', s):
            tiers_values.append(int(re.findall(r'\d+', s)[0]))
          else:
            tiers_values.append('0')
      n_tiers = len(tiers_values)

      # time.sleep(random.lognormal(mean = 0.1, sigma=0.25))
      r = None
      r = rs[1].result()
      result = r.json()
      
      story_html = result["data"]["project"]["story"]
      story = BeautifulSoup(story_html, 'html.parser')
      n_gifs = len(story.find_all('img', {'class': "fit js-lazy-image"}))
      n_images = len(story.find_all('img')) - n_gifs
      

      # text
      story_text = ' '.join([p for p in story.find_all(text=True) if i not in ['\n', ' ']])

      risks = result["data"]["project"]["risks"]

      # time.sleep(random.lognormal(mean = 0.1, sigma=0.25))
      

      df.loc[i] = pd.Series(table.loc[i].values.tolist() + [image, has_video, n_tiers, tiers_values, n_images, n_gifs, websites,
                      fb_linked, n_collab, collab_names], index = cols)
      succesful = True
      if time.time()-page < 2:
        time.sleep(2 - (time.time() - page))
      print('Time for this page was {}s'.format(round(time.time()-page, 2)))
      
      last_5.append(time.time()-page)
      if len(last_5) == 6:
        last_5.pop(0)
        if [x >= 6 for x in last_5] == [True]*5:
          print('Proxy too slow')
          if proxy in used_proxies: used_proxies.remove(proxy) 
          proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
          http_proxy  = 'http://' + proxy
          https_proxy = 'https://' + proxy
          proxyDict = { 
                        "http"  : http_proxy, 
                        "https" : https_proxy, 
                      }
          used_proxies.append(proxy)

    except KeyboardInterrupt:
      k = True
      print('interrupted!')
      break
    
    except:
      if r and r.status_code == 200:
        print('Problems parsing, scraping skipped!!')
        skipped.append(url)
        df.loc[i] = pd.Series(table.loc[i].values.tolist(), index = old_cols)
        succesful =True
      elif r and r.status_code == 429:
        print("Too many requests, rotate ip")
        proxy = proxy_list.pop(random.choice(len(good_proxies))).get_address()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = { 
                      "http"  : http_proxy, 
                      "https" : https_proxy, 
                    }
        used_proxies.append(proxy)
      else:
        print("Bad Proxy")
        if proxy in used_proxies: used_proxies.remove(proxy) 
        proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = { 
                      "http"  : http_proxy, 
                      "https" : https_proxy, 
                    }
        used_proxies.append(proxy)
    
    
  if k:
    break

  
print(f'Total time for {i+1} pages is {round(time.time()-start, 2)} seconds')

------ Page 1: sm4shostunfest/du-sm4sh-au-stunfest-le-20-mai ------
Time for this page was 2.0s
------ Page 2: steveworthington/table-tennis-stick-figure-t-shirt-in-many-colors-p ------
Time for this page was 2.0s
------ Page 3: adriandavila/de-ceniza-y-sal ------
Time for this page was 2.0s
------ Page 4: concordmovie/my-advice-romantic-comedy-feature-length-movie-90 ------
Time for this page was 2.0s
------ Page 5: 1908500678/clothing-label-with-a-90s-state-of-mind-made-in-fr ------
Time for this page was 2.0s
------ Page 6: 230068012/represent-a-feature-length-documentary ------
Time for this page was 2.0s
------ Page 7: 1529991300/the-reel-snow-white ------
Time for this page was 2.0s
------ Page 8: 666837967/fair-made-fancy-pants-fashion-with-passion ------
Time for this page was 2.0s
------ Page 9: 518123151/rivalrys ------
Time for this page was 2.0s
------ Page 10: supermcgamer/money-making-game ------
Time for this page was 2.0s
------ Page 11: 1406415891/montana-williams-sear

In [5]:
df.to_csv(name[:name.find('.csv')] + '_scraped.csv')